In [ ]:
from pathlib import Path

import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.time import Time
from astropy.timeseries import TimeSeries

from matplotlib import pyplot as plt

from stellarphot.differential_photometry import vsx_mags


In [ ]:
folder_with_files = '.'

your_photometry_file = 'some_name.csv'
comp_stars_file = 'apass-V0533-Her.csv'
name_of_variable = 'v0533 her'

# Enter the check star label (2 or 3 digit number) below 
check_star_label = '153'

# Our filter names vs filter names in comparison star table
filter_mapping = dict(
    ip='SI',
    B='B'
)

In [ ]:
p = Path(folder_with_files)
your_photometry_file = p / your_photometry_file
comp_stars_file = p / comp_stars_file

In [ ]:
var_coord = SkyCoord.from_name(name_of_variable)

vc = dict(coords=var_coord)

In [ ]:
your_photometry = Table.read(your_photometry_file)
your_photometry['RA'].unit = 'degree'
your_photometry['Dec'].unit = 'degree'

your_photometry['band'] = your_photometry['filter']

comp_stars = Table.read(comp_stars_file)
comp_stars['RAJ2000'].unit = 'degree'
comp_stars['DEJ2000'].unit = 'degree'



In [ ]:
comp_stars_check_only = comp_stars[comp_stars['label'] == int(check_star_label)]
check_star_auid = comp_stars_check_only['auid'][0]

In [ ]:
comp_stars_no_check = comp_stars[comp_stars['auid'] != check_star_auid]

## The cell below does just one filter...

In [ ]:
this_filter = 'ip'

this_phot = your_photometry[your_photometry['band'] == this_filter]
                                                                                                                                                                                                                                                                                                                                                                                              
this_phot_grp = this_phot.group_by('BJD')

airmass = []
times = []
cal_mag = []
cal_mag_err = []
for time, rows in zip(this_phot_grp.groups.keys, this_phot_grp.groups):
    mag, err = vsx_mags.calc_vmag(vc, rows, comp_stars_no_check, band=filter_mapping[this_filter], star_data_mag_column=f'mag_inst')
    cal_mag.append(mag)
    cal_mag_err.append(err)
    times.append(time[0])
    airmass.append(rows['airmass'][0])
    

In [ ]:
table = Table(data=[times, cal_mag, cal_mag_err, airmass], names=['time', 'mag', 'err', 'airmass'])
table['day']  = [np.floor(bjd) for bjd in table['time']]

table = table.group_by('day')

table_grouped = table

In [ ]:
for group in table_grouped.groups: 
    time0 = group['time'][0]
    plt.plot(label = group['day'])
    plt.errorbar(group['time']-time0,group['mag'], yerr=group['err'], fmt = 's', label = f'{time0:.2f}')
plt.grid()
plt.ylabel('Magnitude')
plt.xlabel('Time since begining of observation (Days)')
plt.legend()
plt.ylim(*plt.ylim()[::-1])


In [ ]:
check = comp_stars[comp_stars['auid'] == check_star_auid]
check_coord = SkyCoord(ra=check['RAJ2000'][0], dec=check['DEJ2000'][0], unit='degree')

In [ ]:
your_coords = SkyCoord(ra=your_photometry['RA'], dec=your_photometry['Dec'])

In [ ]:
kc = dict(coords=check_coord)

airmass_k = []
times_k = []
cal_mag_k = []
cal_mag_err_k = []
for time, rows in zip(this_phot_grp.groups.keys, this_phot_grp.groups):
    mag, err = vsx_mags.calc_vmag(kc, rows, comp_stars_no_check, band=filter_mapping[this_filter], star_data_mag_column=f'mag_inst')
    cal_mag_k.append(mag)
    cal_mag_err_k.append(err)
    times_k.append(time[0])
    airmass_k.append(rows['airmass'][0])
    
assert times_k == times

In [ ]:
table_k = Table(data=[times_k, cal_mag_k, cal_mag_err_k, airmass_k], names=['time', 'mag', 'err', 'airmass'])
table_k['day']  = table['day']

table_k = table_k.group_by('day')

table_grouped_k = table_k

In [ ]:
for group in table_grouped_k.groups: 
    time0 = group['time'][0]
    plt.plot(label = group['day'])
    plt.errorbar(group['time']-time0,group['mag'], yerr=group['err'], fmt = 's', label = f'{time0:.2f}')
plt.grid()
plt.ylabel('Magnitude')
plt.xlabel('Time since begining of observation (Days)')
plt.title('Check star')
plt.legend()
plt.ylim(15.2, 14.7)
